# Build an Tweet BERT Classifier Pipeline using Kubeflow and SageMaker

In [103]:
! pip install boto3

## Install Kubeflow Pipelines SDK

In [104]:
# !pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade

In [105]:
import boto3

#################################
#################################
# REPLACE AWS_REGION= with the current region
#  surround with single quotes
AWS_REGION='us-west-2'

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')
print('Account ID: {}'.format(AWS_ACCOUNT_ID))

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

Account ID: 057716757052
S3 Bucket: sagemaker-us-west-2-057716757052


# Build Pipeline 

## 1. Run the following command to load Kubeflow Pipelines SDK

In [106]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

## 2.Load reusable sagemaker components

In [107]:
# sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/train/component.yaml')
sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/cb36f87b727df0578f4c1e3fe9c24a30bb59e5a2/components/aws/sagemaker/train/component.yaml')
sagemaker_model_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/model/component.yaml')
sagemaker_deploy_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/deploy/component.yaml')


In [108]:
channels='[ \
                    { \
                        "ChannelName": "train", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "s3://'+S3_BUCKET+'/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-train", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    }, \
                    { \
                        "ChannelName": "validation", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "s3://'+S3_BUCKET+'/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-validation", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    }, \
                    { \
                        "ChannelName": "test", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "s3://'+S3_BUCKET+'/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-test", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    } \
                ]'

In [109]:
# epochs=1
# train_steps_per_epoch=100

# max_seq_length = 128
# learning_rate=0.00001
# epsilon=0.00000001
# train_batch_size=128
# validation_batch_size=128
# test_batch_size=128

# # train_steps_per_epoch=10
# validation_steps=100
# test_steps=100

# train_instance_count=2 # modified by gonsoo
# train_instance_type='ml.p3.2xlarge'
# train_volume_size=1024

# use_xla=True
# use_amp=True
# freeze_bert_layer=False
# enable_checkpointing=True
# input_mode='Pipe'

In [119]:
epochs= "1"
train_steps_per_epoch= "1000"

max_seq_length = "128"
learning_rate= "0.00001"
epsilon= "0.00000001"
train_batch_size= "128"
validation_batch_size= "128"
test_batch_size= "128"

# train_steps_per_epoch=10
validation_steps= "100"
test_steps= "100"

train_instance_count= "2" 
train_instance_type='ml.p3.2xlarge'
train_volume_size= "1024"

use_xla= "True"
use_amp= "True"
freeze_bert_layer= "False"
enable_checkpointing= "True"
input_mode='Pipe'

## 3.Create Pipeline

In [120]:
SAGEMAKER_ROLE_ARN = 'arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20191128T110038'
# Configure your s3 bucket
# S3_PIPELINE_PATH= 's3://{}/'.format(S3_BUCKET)
# processed_train_data_s3_uri = 's3://sagemaker-us-west-2-057716757052/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-train'
# processed_validation_data_s3_uri = 's3://sagemaker-us-west-2-057716757052/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-validation'
# processed_test_data_s3_uri = 's3://sagemaker-us-west-2-057716757052/sagemaker-scikit-learn-2020-06-28-05-08-39-660/output/bert-test'

if AWS_REGION == 'us-west-2':
    AWS_ECR_REGISTRY = "057716757052.dkr.ecr.us-west-2.amazonaws.com/bert2tweet:latest"
    
model_output_path = 's3://sagemaker-us-west-2-057716757052/sagemaker-scikit-learn-2020-06-28-05-08-39-660/model'

In [121]:
@dsl.pipeline(
    name='Tweet BERT Classification pipeline',
    description='Tweet BERT Classification using KMEANS in SageMaker'
)
def tweet_BERT(
    region = AWS_REGION,
    image = AWS_ECR_REGISTRY,
    dataset_path = channels,
    instance_type = 'ml.p3.8xlarge',
    instance_count = 2,
    volume_size = '50',
    model_putput_path = model_output_path,
    role_arn = SAGEMAKER_ROLE_ARN,
    network_isolation='False',
    traffic_encryption='False',
    spot_instance='False'    
    ):
    # Component 1
    training = sagemaker_train_op(
        region = region,
        image = image,
        channels=channels,        
        instance_type = instance_type,
        instance_count = instance_count,
        volume_size = volume_size,
        model_artifact_path=model_output_path,
        role=role_arn,
        network_isolation=network_isolation,
        traffic_encryption=traffic_encryption,
        spot_instance=spot_instance,        
        hyperparameters={'epochs': epochs,
                        'learning_rate': learning_rate,
                        'epsilon': epsilon,
                        'train_batch_size': train_batch_size,
                        'validation_batch_size': validation_batch_size,
                        'test_batch_size': test_batch_size,                                             
                        'train_steps_per_epoch': train_steps_per_epoch,
                        'validation_steps': validation_steps,
                        'test_steps': test_steps,
                        'use_xla': use_xla,
                        'use_amp': use_amp,                                             
                        'max_seq_length': max_seq_length,
                        'freeze_bert_layer': freeze_bert_layer,
                        'enable_checkpointing': enable_checkpointing
                        },        
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))        
    # Component 2
#     create_model = sagemaker_model_op(
#         region = region,
#         model_name = training.outouts['job_name'],
#         image = 
#     )
        


        


In [122]:
kfp.compiler.Compiler().compile(tweet_BERT, 'tweet_BERT.zip')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/kfp/components/_data_passing.py:94: UserWarning: Missing type name was inferred as "JsonObject" based on the value "{'epochs': '1', 'learning_rate': '0.00001', 'epsilon': '0.00000001', 'train_batch_size': '128', 'validation_batch_size': '128', 'test_batch_size': '128', 'train_steps_per_epoch': '1000', 'validation_steps': '100', 'test_steps': '100', 'use_xla': 'True', 'use_amp': 'True', 'max_seq_length': '128', 'freeze_bert_layer': 'False', 'enable_checkpointing': 'True'}".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


In [123]:
!ls -al ./tweet_BERT.zip

-rw-rw-r-- 1 ec2-user ec2-user 1970 Jul  5 10:25 ./tweet_BERT.zip


In [124]:
!unzip -o ./tweet_BERT.zip

Archive:  ./tweet_BERT.zip
  inflating: pipeline.yaml           


In [125]:
# !cat pipeline.yaml

In [126]:
import time

In [127]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')

exp_name    = f'tweet-BERT-train-deploy-kfp-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
my_run = client.run_pipeline(aws_experiment.id, exp_name, 'tweet_BERT.zip')

In [93]:
#     training = sagemaker_train_op(
#         region=region,
#         image=train_image,
#         training_input_mode=training_input_mode,
#         hyperparameters=training_hyp.output,
#         channels=channels,
#         instance_type=instance_type,
#         instance_count='1',
#         volume_size=volume_size,
#         max_run_time=max_run_time,
#         model_artifact_path=f's3://{bucket_name}/jobs',
#         network_isolation=network_isolation,
#         traffic_encryption=traffic_encryption,
#         spot_instance=spot_instance,
#         role=role,
#     )